In [13]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import keras.callbacks
from keras import optimizers
import argparse
import pickle
from RNN_utils import *



In [14]:
DEFAULT_LAYERS = 2
DEFAULT_HIDDEN = 500
DEFAULT_DROPOUT = 0.0

layers = [1, 3]
hidden = [300, 700]
dropout = [0.15, 0.3]
DATA_DIR = './data/shakespeare_input.txt'
BATCH_SIZE = 30
SEQ_LENGTH = 50

num_epochs = 50

# Creating training data
X, y, VOCAB_SIZE, ix_to_char, char_to_ix = load_data(DATA_DIR, SEQ_LENGTH)

Data length: 4573338 characters
Vocabulary size: 67 characters


In [15]:
# Split testing data from training/validation data
train_split = 0.8
test_ind = int(round(train_split*len(X)))
X_test = X[test_ind:]
y_test = y[test_ind:]
X_train = X[:test_ind]
y_train = y[:test_ind]

In [16]:
def make_lstm_model(VOCAB_SIZE, num_layers, num_hidden, dropout):
    model = Sequential()
    model.add(LSTM(num_hidden, input_shape=(None, VOCAB_SIZE), return_sequences=True, dropout=dropout))
    for i in range(num_layers - 1):
      model.add(LSTM(num_hidden, return_sequences=True,dropout=dropout))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))
    model.add(Activation('softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    return model

In [17]:
def run_model(model, X, y, num_epochs=50, batch_size=30):
    early_stop1 = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=1e-3,
                              patience=2,
                              verbose=1, mode='auto')
    early_stop2 = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=1e-3,
                              patience=2,
                              verbose=1, mode='auto')
    history = model.fit(X, y, batch_size=batch_size, callbacks=[early_stop1, early_stop2], validation_split=0.2, epochs=num_epochs, verbose=1)
    return model, history.history

In [18]:
def save_model(model, history, num_layers, num_hidden, dropout):
    string = 'layers_{}_hidden_{}_dropout_{}_epoch_{}'.format(num_layers, num_hidden, int(dropout*10), len(history['loss']))
    print('saving: ' + string)
    with open('history_'+string, 'wb') as file_pi:
        pickle.dump(history, file_pi)
    model.save_weights('weights_'+string+'.hdf5')

In [7]:
default_model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, DEFAULT_HIDDEN, DEFAULT_DROPOUT)

In [19]:
generate_text(default_model, 100, VOCAB_SIZE, ix_to_char, -1, 1)

refore,
'Riest Soly Bricher and, a land shall,
On every piece of the off the poison.

GLOUCESTER:
Lo

"refore,\n'Riest Soly Bricher and, a land shall,\nOn every piece of the off the poison.\n\nGLOUCESTER:\nLoo"

In [9]:
default_model, default_history = run_model(default_model, X_train, y_train)
save_model(default_model, default_history, DEFAULT_LAYERS, DEFAULT_HIDDEN, DEFAULT_DROPOUT)

Train on 58538 samples, validate on 14635 samples
Epoch 1/50
58538/58538 [==============================] - 746s 13ms/step - loss: 1.9000 - val_loss: 1.7130
Epoch 2/50
58538/58538 [==============================] - 746s 13ms/step - loss: 1.4438 - val_loss: 1.6091
Epoch 3/50
58538/58538 [==============================] - 741s 13ms/step - loss: 1.3576 - val_loss: 1.5810
Epoch 4/50
58538/58538 [==============================] - 750s 13ms/step - loss: 1.3096 - val_loss: 1.5694
Epoch 5/50
58538/58538 [==============================] - 748s 13ms/step - loss: 1.2742 - val_loss: 1.5685
Epoch 6/50
58538/58538 [==============================] - 745s 13ms/step - loss: 1.2442 - val_loss: 1.5691
Epoch 00006: early stopping


In [10]:
for layer in layers:
    model = make_lstm_model(VOCAB_SIZE, layer, DEFAULT_HIDDEN, DEFAULT_DROPOUT)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, layer, DEFAULT_HIDDEN, DEFAULT_DROPOUT)
    

Train on 58538 samples, validate on 14635 samples
Epoch 1/50
58538/58538 [==============================] - 305s 5ms/step - loss: 2.0653 - val_loss: 1.8772
Epoch 2/50
58538/58538 [==============================] - 304s 5ms/step - loss: 1.5839 - val_loss: 1.7046
Epoch 3/50
58538/58538 [==============================] - 305s 5ms/step - loss: 1.4603 - val_loss: 1.6520
Epoch 4/50
58538/58538 [==============================] - 304s 5ms/step - loss: 1.3983 - val_loss: 1.6256
Epoch 5/50
58538/58538 [==============================] - 302s 5ms/step - loss: 1.3586 - val_loss: 1.6112
Epoch 6/50
58538/58538 [==============================] - 302s 5ms/step - loss: 1.3293 - val_loss: 1.6039
Epoch 7/50
58538/58538 [==============================] - 302s 5ms/step - loss: 1.3059 - val_loss: 1.5968
Epoch 8/50
58538/58538 [==============================] - 302s 5ms/step - loss: 1.2865 - val_loss: 1.5993
Epoch 9/50
58538/58538 [==============================] - 302s 5ms/step - loss: 1.2694 - val_loss: 1.6

In [11]:
for h in hidden:
    model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, h, DEFAULT_DROPOUT)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, DEFAULT_LAYERS, h, DEFAULT_DROPOUT)
    

Train on 58538 samples, validate on 14635 samples
Epoch 1/50
58538/58538 [==============================] - 347s 6ms/step - loss: 2.0082 - val_loss: 1.7885
Epoch 2/50
58538/58538 [==============================] - 332s 6ms/step - loss: 1.5321 - val_loss: 1.6606
Epoch 3/50
58538/58538 [==============================] - 332s 6ms/step - loss: 1.4277 - val_loss: 1.6209
Epoch 4/50
58538/58538 [==============================] - 330s 6ms/step - loss: 1.3763 - val_loss: 1.6035
Epoch 5/50
58538/58538 [==============================] - 330s 6ms/step - loss: 1.3424 - val_loss: 1.5834
Epoch 6/50
58538/58538 [==============================] - 332s 6ms/step - loss: 1.3175 - val_loss: 1.5839
Epoch 7/50
58538/58538 [==============================] - 333s 6ms/step - loss: 1.2971 - val_loss: 1.5801
Epoch 8/50
58538/58538 [==============================] - 334s 6ms/step - loss: 1.2802 - val_loss: 1.5762
Epoch 9/50
58538/58538 [==============================] - 332s 6ms/step - loss: 1.2653 - val_loss: 1.5

In [ ]:
for d in dropout:
    model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, DEFAULT_HIDDEN, d)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, DEFAULT_LAYERS, DEFAULT_HIDDEN, d)
    

Train on 58538 samples, validate on 14635 samples
Epoch 1/50
58538/58538 [==============================] - 758s 13ms/step - loss: 2.0839 - val_loss: 1.7685
Epoch 2/50
58538/58538 [==============================] - 761s 13ms/step - loss: 1.5982 - val_loss: 1.6469
Epoch 3/50
58538/58538 [==============================] - 750s 13ms/step - loss: 1.4735 - val_loss: 1.6068
Epoch 4/50
58538/58538 [==============================] - 750s 13ms/step - loss: 1.4124 - val_loss: 1.5773
Epoch 5/50
58538/58538 [==============================] - 755s 13ms/step - loss: 1.3750 - val_loss: 1.5613
Epoch 6/50
58538/58538 [==============================] - 756s 13ms/step - loss: 1.3477 - val_loss: 1.5537
Epoch 7/50
58538/58538 [==============================] - 761s 13ms/step - loss: 1.3268 - val_loss: 1.5610
Epoch 8/50
58538/58538 [==============================] - 762s 13ms/step - loss: 1.3101 - val_loss: 1.5426
Epoch 9/50
58538/58538 [==============================] - 753s 13ms/step - loss: 1.2959 - val_

In [ ]:
# just run second dropout because it got stuck for some reason
model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, DEFAULT_HIDDEN, dropout[1])
model, history = run_model(model, X_train, y_train)
history['test_loss_predict'] = evaluate_loss(model, X_test)
history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
save_model(model, history, DEFAULT_LAYERS, DEFAULT_HIDDEN, dropout[1])
    

Train on 58538 samples, validate on 14635 samples
Epoch 1/50
58538/58538 [==============================] - 767s 13ms/step - loss: 2.2032 - val_loss: 1.8457
Epoch 2/50
58538/58538 [==============================] - 765s 13ms/step - loss: 1.6906 - val_loss: 1.7047
Epoch 3/50
58538/58538 [==============================] - 766s 13ms/step - loss: 1.5517 - val_loss: 1.6432
Epoch 4/50
58538/58538 [==============================] - 762s 13ms/step - loss: 1.4843 - val_loss: 1.6049
Epoch 5/50
58538/58538 [==============================] - 763s 13ms/step - loss: 1.4434 - val_loss: 1.5945
Epoch 6/50
58538/58538 [==============================] - 762s 13ms/step - loss: 1.4149 - val_loss: 1.5806
Epoch 7/50
 2040/58538 [>.............................] - ETA: 11:38 - loss: 1.3885